In [2]:
import pandas as pd

In [2]:
#df = pd.read_csv('/Users/AuerPower/Metis/git/predict_gentrification_with_yelp/data/raw_zillow/Neighborhood_Zhvi_AllHomes.csv')
df = pd.read_csv('/Users/AuerPower/Metis/git/predict_gentrification_with_yelp/data/raw_zillow/Zip_Zhvi_AllHomes.csv',
                 encoding="ISO-8859-1")
df.head(6)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,229011.0,229723.0,230303.0,...,1323473,1310806,1304288,1292400,1268453,1231999,1200225,1190513,1201389,1212916
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,204066.0,203752.0,204737.0,...,484536,485356,484823,483136,482049,479947,477869,475778,474614,473836
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,247266.0,248723.0,249675.0,...,1182445,1174980,1174939,1171951,1176050,1170613,1145540,1136414,1147577,1178571
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,197780.0,196728.0,195786.0,...,335615,334863,334371,334005,334189,333952,334281,334285,334505,334135
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,275535.0,275100.0,276134.0,...,631204,632961,632124,629721,627803,625515,623036,620855,619097,617942
5,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,6,97347.0,97155.0,96978.0,...,182587,182597,182968,183577,184509,185282,185546,186079,187038,187958


In [3]:
# Get names of date columns for selection
date_names = df.columns[7:]
date_names

Index(['1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09',
       '1996-10', '1996-11', '1996-12', '1997-01',
       ...
       '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
       '2019-09', '2019-10', '2019-11', '2019-12'],
      dtype='object', length=285)

In [4]:
# Subset data frame and transpose
df_transpose = df[date_names]
df_transpose = df_transpose.transpose()
df_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461
1996-04,229011.0,204066.0,247266.0,197780.0,275535.0,97347.0,NaN,97291.0,82363.0,117723.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-05,229723.0,203752.0,248723.0,196728.0,275100.0,97155.0,NaN,97147.0,82331.0,117715.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-06,230303.0,204737.0,249675.0,195786.0,276134.0,96978.0,NaN,97017.0,82276.0,118417.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-07,229291.0,205312.0,249308.0,197174.0,276491.0,96655.0,NaN,96680.0,82227.0,118963.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-08,230021.0,206137.0,249326.0,197194.0,277210.0,96682.0,NaN,96575.0,82167.0,119884.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# convert index to datetime and run median average per year
df_transpose.index = pd.to_datetime(df_transpose.index, format='%Y-%m')

With time series data, using pad/ffill is extremely common so that the “last known value” is available at every time point.

### The simplest method would be to look at median house price per neighborhood for one timeslice
lets calculate a column for that first

In [6]:
# get average median housing prices for time slice 2015-2019
def median_2015_2019(row):
    year = df_transpose[row].resample('Y', axis='index').mean().ffill()
    year = year[19:].mean()
    return year

In [7]:
df['median_2015_2019'] = df.index.to_series().apply(median_2015_2019).round(2)
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,median_2015_2019
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,229011.0,229723.0,230303.0,...,1310806,1304288,1292400,1268453,1231999,1200225,1190513,1201389,1212916,1262735.00
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,204066.0,203752.0,204737.0,...,485356,484823,483136,482049,479947,477869,475778,474614,473836,472785.98
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,247266.0,248723.0,249675.0,...,1174980,1174939,1171951,1176050,1170613,1145540,1136414,1147577,1178571,1238752.50
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,197780.0,196728.0,195786.0,...,334863,334371,334005,334189,333952,334281,334285,334505,334135,333001.72
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,275535.0,275100.0,276134.0,...,632961,632124,629721,627803,625515,623036,620855,619097,617942,617419.80


### Another metric would be rate of change/growth rate as a proxy for gentrification 
but I'm not sure if this works with the yelp data - would I have to calculate rate of change for those metrics too? Or at least have some time-series information. I'm going to calculate a column for that in case I want to use it. For this one let's do 2010-2019

In [8]:
# Compound growth rate formula - not sure if I need to use this for home price
def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods)) - 1

def ca_growth_rate(row):
    #row = df.index[df['RegionID'] == RegionID]
    year = df_transpose[row].resample('Y', axis='index').mean().ffill()
    year_2010_2019 = year[14:]
    start = float(year_2010_2019.iloc[0])
    end = float(year_2010_2019.iloc[-1])
    periods = len(year_2010_2019)
    gr = cagr(start, end, periods)
    return gr

In [9]:
# Percent change between 2019 and 2010
def growth_rate(row):
    year = df_transpose[row].resample('Y', axis='index').mean().ffill()
    year_2010_2019 = pd.Series([year['2010-12-31'], year['2019-12-31']])
    change = year_2010_2019.pct_change()[1]
    return change

In [10]:
df['change_2010_2019'] = df.index.to_series().apply(growth_rate)
df['avg_growth_rate_2010_2019'] = df.index.to_series().apply(ca_growth_rate)
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,median_2015_2019,change_2010_2019,avg_growth_rate_2010_2019
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,229011.0,229723.0,230303.0,...,1292400,1268453,1231999,1200225,1190513,1201389,1212916,1262735.00,0.425656,0.036100
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,204066.0,203752.0,204737.0,...,483136,482049,479947,477869,475778,474614,473836,472785.98,0.156252,0.014624
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,247266.0,248723.0,249675.0,...,1171951,1176050,1170613,1145540,1136414,1147577,1178571,1238752.50,0.362704,0.031431
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,197780.0,196728.0,195786.0,...,334005,334189,333952,334281,334285,334505,334135,333001.72,0.251599,0.022696
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,275535.0,275100.0,276134.0,...,629721,627803,625515,623036,620855,619097,617942,617419.80,0.182297,0.016887


In [11]:
# reduce columns and save as csv
changes_df = df[['RegionID', 'RegionName', 'City', 'State', 'Metro']].join(
    df[['median_2015_2019', 'change_2010_2019', 'avg_growth_rate_2010_2019']])
changes_df.to_csv('data/house_price_changes.csv')

### Zillow has data on affordabiliy but it's by City not by neighborhood

In [3]:
affordability = pd.read_csv('data/Affordability_Wide_2019Q3_Public.csv')
affordability[affordability['RegionName'] == 'San Francisco, CA']

,RegionID,RegionName,SizeRank,Index,HistoricAverage_1985thru1999,1979-03,1979-06,1979-09,1979-12,1980-03,...,2017-06,2017-09,2017-12,2018-03,2018-06,2018-09,2018-12,2019-03,2019-06,2019-09
11,395057,"San Francisco, CA",11,Price To Income,5.002622,4.253145,4.382501,4.572843,4.703875,4.834304,...,8.558171,8.555372,8.613288,8.951910,9.064145,9.200361,9.199402,9.133212,9.027692,8.966299
380,395057,"San Francisco, CA",11,Mortgage Affordability,0.384347,0.371355,0.402889,0.428045,0.496002,0.597224,...,0.387515,0.383166,0.392383,0.432379,0.444523,0.454370,0.454852,0.431841,0.403826,0.391827
749,395057,"San Francisco, CA",11,Rent Affordability,0.312387,0.289501,0.287501,0.284881,0.282976,0.279761,...,0.368852,0.366968,0.356964,0.354777,0.356849,0.358575,0.357654,0.358805,0.362604,0.364676


In [4]:
affordability['RegionName'].unique()

array(['United States', 'New York, NY',
       'Los Angeles-Long Beach-Anaheim, CA', 'Chicago, IL',
       'Dallas-Fort Worth, TX', 'Philadelphia, PA', 'Houston, TX',
       'Washington, DC', 'Miami-Fort Lauderdale, FL', 'Atlanta, GA',
       'Boston, MA', 'San Francisco, CA', 'Detroit, MI', 'Riverside, CA',
       'Phoenix, AZ', 'Seattle, WA', 'Minneapolis-St Paul, MN',
       'San Diego, CA', 'St. Louis, MO', 'Tampa, FL', 'Baltimore, MD',
       'Denver, CO', 'Pittsburgh, PA', 'Portland, OR', 'Charlotte, NC',
       'Sacramento, CA', 'San Antonio, TX', 'Orlando, FL',
       'Cincinnati, OH', 'Cleveland, OH', 'Kansas City, MO',
       'Las Vegas, NV', 'Columbus, OH', 'Indianapolis, IN',
       'San Jose, CA', 'Austin, TX', 'Virginia Beach, VA',
       'Nashville, TN', 'Providence, RI', 'Milwaukee, WI',
       'Jacksonville, FL', 'Memphis, TN', 'Oklahoma City, OK',
       'Louisville-Jefferson County, KY', 'Hartford, CT', 'Richmond, VA',
       'New Orleans, LA', 'Buffalo, NY', 'Raleig